In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data,nb.scraps['TabNet MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost", "TabNet"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])
tabnet = median(baseline_data[:,2])
print(stats)
print(catboost)
print(tabnet)

0.3759573324557224
0.21415930265621955


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.086476,0.219009,2.969888,22.338930
1,0.074879,0.218798,2.763604,22.317361
2,0.152364,0.249805,3.942204,25.480158
3,0.342846,0.322518,5.913549,32.896806
4,0.132134,0.231444,3.671164,23.607301
5,0.169724,0.256185,4.160579,26.130891
6,0.126452,0.247890,3.591376,25.284811
7,0.107443,0.223950,3.310443,22.842940
8,0.114024,0.234076,3.410323,23.875752
9,0.129341,0.259373,3.632137,26.456039


MEDIAN:
MSE                    0.127897
MAE                    0.240983
Euclidean Distance     3.611757
Manhattan Distance    24.580282
dtype: float64
0.24098315263406245


# GAN Model with TabNet generator

In [ ]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps["GAN_2 Metrics"].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[1,0.1],[1,0.01],[1,0],[0.1,1],[0.1,0.1],[0.1,0.01],[0.1,0],[0.01,1],[0.01,0.1],[0.01,0.01],[0.01,0]]
abc_mae = [[] for i in range(12)]
abc_tab_mae = [[] for i in range(12)]
abc_mae_skip = [[] for i in range(12)]
abc_tab_mae_skip = [[] for i in range(12)]
abc_mae_mean = [[] for i in range(12)]
abc_mae_skip_mean = [[] for i in range(12)]
abc_weights = [[] for i in range(12)]
prior_model = [[] for i in range(12)]
abc_pre_generator = [[] for i in range(12)]

for nb in book.notebooks:
    # metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics2 = np.array(nb.scraps['ABC_GAN_2 Metrics'].data)
    # metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    # metrics4 = np.array(nb.scraps['ABC_GAN_4 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(12):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                # abc_mae[i].append(metrics1[1,j])
                # abc_mae_skip[i].append(metrics3[1,j])
                abc_tab_mae[i].append(metrics2[1,j])
                # abc_tab_mae_skip[i].append(metrics4[1,j])
            # abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MAE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MAE'].data)
            # abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            # abc_mae_mean[i].append(mean(metrics1[1,:]))
            abc_mae_mean[i].append(mean(metrics2[1,:]))

In [6]:

data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.23871,1.20307,0.76774,0.23441,0.26580
1,1,1,0.21123,1.17292,0.18784,0.21027,0.22212
2,1,1,0.22955,1.24175,0.24603,0.22932,0.22579
3,1,1,0.19635,1.21986,0.19093,0.21700,0.20445
4,1,1,0.19293,1.04122,0.58931,0.24263,0.21756
5,1,1,0.25532,1.13739,0.98736,0.30822,0.27420
6,1,1,0.22392,1.16986,0.99415,0.24278,0.22081
7,1,1,0.22811,1.27994,0.29041,0.22321,0.27405
8,1,1,0.23392,1.20611,0.99130,0.25836,0.29057
9,1,1,0.25567,1.36143,0.21787,0.26401,0.29959


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.228833
ABC pre-generator MAE            1.204591
Skip Node weight                 0.439860
ABC GAN MAE                      0.238518
ABC_GAN MAE (skip connection)    0.245797
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.1,0.19933,0.96097,0.73907,0.21025,0.19920
1,1,0.1,0.19164,1.00754,0.70637,0.20157,0.19377
2,1,0.1,0.25590,0.94953,0.13906,0.28442,0.27195
3,1,0.1,0.22187,0.98943,0.13969,0.21060,0.23155
4,1,0.1,0.18911,1.04031,0.14918,0.20704,0.19588
5,1,0.1,0.19444,1.00212,0.14234,0.22219,0.22994
6,1,0.1,0.21308,1.02979,0.16744,0.21222,0.21897
7,1,0.1,0.23929,1.04466,0.15430,0.26060,0.25430
8,1,0.1,0.21019,1.02679,0.54730,0.22549,0.22366
9,1,0.1,0.27169,1.05944,0.62663,0.26459,0.23974


Variance                         1.000000
Bias                             0.100000
Prior Model MAE                  0.211634
ABC pre-generator MAE            1.017164
Skip Node weight                 0.160869
ABC GAN MAE                      0.217207
ABC_GAN MAE (skip connection)    0.226800
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.01,0.21607,0.99484,0.13226,0.22093,0.23888
1,1,0.01,0.19369,1.01180,0.12638,0.19941,0.19279
2,1,0.01,0.23937,0.99642,0.11462,0.25263,0.24581
3,1,0.01,0.19844,0.99770,0.33113,0.23388,0.19620
4,1,0.01,0.23986,1.04156,0.15345,0.26245,0.23598
5,1,0.01,0.23264,1.04354,0.11556,0.24157,0.24093
6,1,0.01,0.20071,0.98872,0.27368,0.21441,0.21284
7,1,0.01,0.19088,1.01807,0.11478,0.21151,0.21189
8,1,0.01,0.24134,0.95823,0.43189,0.24454,0.27111
9,1,0.01,0.18339,0.99348,0.28138,0.19299,0.19696


...

Variance                         0.010000
Bias                             1.000000
Prior Model MAE                  0.206576
ABC pre-generator MAE            0.812001
Skip Node weight                 0.598147
ABC GAN MAE                      0.238558
ABC_GAN MAE (skip connection)    0.233077
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.1,0.25426,0.26274,0.31940,0.25864,0.25414
1,0.01,0.1,0.22003,0.24000,0.25370,0.19618,0.21583
2,0.01,0.1,0.19315,0.20426,0.48788,0.20590,0.20346
3,0.01,0.1,0.16926,0.19299,0.46594,0.18346,0.17840
4,0.01,0.1,0.23033,0.24572,0.01987,0.24375,0.25018
5,0.01,0.1,0.19085,0.20109,0.24454,0.20023,0.18036
6,0.01,0.1,0.25198,0.27032,0.12363,0.28139,0.28016
7,0.01,0.1,0.19897,0.22591,0.00000,0.19798,0.21973
8,0.01,0.1,0.22291,0.23892,0.33069,0.20938,0.22718
9,0.01,0.1,0.22703,0.24486,0.39684,0.22816,0.22194


Variance                         0.010000
Bias                             0.100000
Prior Model MAE                  0.221470
ABC pre-generator MAE            0.239464
Skip Node weight                 0.286548
ABC GAN MAE                      0.207638
ABC_GAN MAE (skip connection)    0.220833
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.22722,0.22838,0.00000,0.24146,0.22898
1,0.01,0.01,0.23893,0.23973,0.02996,0.27400,10.87423
2,0.01,0.01,0.20404,0.20205,0.00000,0.23595,0.20345
3,0.01,0.01,0.23129,0.23105,0.13708,0.23388,0.22271
4,0.01,0.01,0.21878,0.21997,0.06872,0.23084,0.23371
5,0.01,0.01,0.25798,0.25946,0.00000,0.28546,0.25932
6,0.01,0.01,0.20597,0.20874,0.00000,0.20594,0.20829
7,0.01,0.01,0.23122,0.23148,0.13571,0.23828,0.24272
8,0.01,0.01,0.24300,0.24516,0.09258,0.25677,0.26161
9,0.01,0.01,0.22652,0.22536,0.09204,0.24039,0.22885


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.229221
ABC pre-generator MAE            0.229717
Skip Node weight                 0.049339
ABC GAN MAE                      0.239337
ABC_GAN MAE (skip connection)    0.231348
dtype: float64
-------------------------------------------------------------


In [7]:
# Display Catboost Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([gan_median,catboost, paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['GAN','catboost','Variance','Bias','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,GAN,catboost,Variance,Bias,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,0.24098,0.21416,1.00,1.00,1.20459,0.23852,0.24580,0.43986
1,0.24098,0.21416,1.00,0.10,1.01716,0.21721,0.22680,0.16087
2,0.24098,0.21416,1.00,0.01,0.99706,0.22741,0.22441,0.14285
3,0.24098,0.21416,0.10,1.00,0.85781,0.23765,0.24044,0.93295
4,0.24098,0.21416,0.10,0.10,0.25252,0.21790,0.23349,0.26294
5,0.24098,0.21416,0.10,0.01,0.23968,0.22659,0.21426,0.08802
6,0.24098,0.21416,0.01,1.00,0.81200,0.23856,0.23308,0.59815
7,0.24098,0.21416,0.01,0.10,0.23946,0.20764,0.22083,0.28655
8,0.24098,0.21416,0.01,0.01,0.22972,0.23934,0.23135,0.04934


## ABC Pre-generator - Stats 


In [8]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[0.1,1],[0.01,1],[1,0.1],[0.1,0.1],[0.01,0.1],[1,0.01],[0.1,0.01],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [9]:
data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.38441,1.19425,0.99096,0.25058,0.24063
1,1,1,0.35555,1.15000,0.99560,0.29397,0.25009
2,1,1,0.37367,1.34448,0.99927,0.24095,0.21568
3,1,1,0.34453,1.44695,0.98646,0.44769,0.20355
4,1,1,0.37024,1.41781,0.98842,0.26225,0.27517
5,1,1,0.36662,1.27891,0.99568,0.28324,0.28622
6,1,1,0.35475,1.16425,0.99773,0.25336,0.25868
7,1,1,0.41523,1.24944,0.99218,0.24734,0.30383
8,1,1,0.33215,1.36950,0.99107,0.28349,0.27437
9,1,1,0.38747,1.31650,0.99026,0.29052,0.22455


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.368431
ABC pre-generator MAE            1.297704
Skip Node weight                 0.991625
ABC GAN MAE                      0.272744
ABC_GAN MAE (skip connection)    0.254383
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,1,0.41320,0.97039,0.98715,0.39015,0.22730
1,0.1,1,0.40012,0.77289,0.97988,0.26334,0.25393
2,0.1,1,0.32547,0.92152,0.98686,0.23259,0.21882
3,0.1,1,0.39824,0.89129,0.98073,0.28564,0.26323
4,0.1,1,0.43269,0.89868,1.00000,0.33791,0.29760
5,0.1,1,0.39555,0.97747,0.99532,0.27250,0.27532
6,0.1,1,0.43413,0.89964,1.00000,0.28515,0.30212
7,0.1,1,0.37551,1.02392,0.98788,0.26116,0.24363
8,0.1,1,0.34992,0.95982,0.99976,0.30776,0.26529
9,0.1,1,0.32741,0.86758,0.99135,0.22877,0.23881


Variance                         0.100000
Bias                             1.000000
Prior Model MAE                  0.396895
ABC pre-generator MAE            0.910582
Skip Node weight                 0.989615
ABC GAN MAE                      0.278823
ABC_GAN MAE (skip connection)    0.258579
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,1,0.40186,0.90927,0.98487,0.23326,0.22815
1,0.01,1,0.33940,0.82411,0.98580,0.23671,0.37412
2,0.01,1,0.36877,0.84407,0.98429,0.26444,0.21792
3,0.01,1,0.39334,0.78604,0.98693,0.27271,0.26710
4,0.01,1,0.30121,0.78573,0.96945,0.23417,0.27259
5,0.01,1,0.41867,1.01484,0.98884,0.30302,0.26585
6,0.01,1,0.37551,0.96280,0.99290,0.26235,0.28648
7,0.01,1,0.35128,0.84670,0.99394,0.25282,0.28224
8,0.01,1,0.37352,0.86473,0.99135,0.26297,0.25851
9,0.01,1,0.39591,0.83111,0.99411,0.27147,0.32044


...

Variance                         1.000000
Bias                             0.010000
Prior Model MAE                  0.346285
ABC pre-generator MAE            1.040430
Skip Node weight                 0.331868
ABC GAN MAE                      0.278761
ABC_GAN MAE (skip connection)    0.282704
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.01,0.32214,0.35894,0.24609,0.23856,0.23074
1,0.1,0.01,0.35277,0.39034,0.18309,0.28026,0.28669
2,0.1,0.01,0.39720,0.41340,0.06963,0.23659,20.05421
3,0.1,0.01,0.38324,0.40926,0.32506,0.37247,0.33210
4,0.1,0.01,0.34837,0.38479,0.23038,0.23655,0.27277
5,0.1,0.01,0.41402,0.42968,0.20098,0.39060,129.94508
6,0.1,0.01,0.32459,0.36818,0.13832,0.19343,0.25700
7,0.1,0.01,0.39174,0.42402,0.19690,0.27419,0.28502
8,0.1,0.01,0.31997,0.34754,0.15323,0.28407,0.25298


Variance                         0.100000
Bias                             0.010000
Prior Model MAE                  0.352772
ABC pre-generator MAE            0.390342
Skip Node weight                 0.196897
ABC GAN MAE                      0.274191
ABC_GAN MAE (skip connection)    0.285024
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.35417,0.35413,0.14305,0.22426,0.27522
1,0.01,0.01,0.39200,0.39570,0.15264,0.26847,0.23944
2,0.01,0.01,0.36703,0.36967,0.12541,0.25029,0.24318
3,0.01,0.01,0.43464,0.43632,0.16226,0.29425,0.28818
4,0.01,0.01,0.37859,0.38103,0.18956,0.31401,0.29984
5,0.01,0.01,0.37624,0.37791,0.27690,0.36500,0.33988
6,0.01,0.01,0.36388,0.36567,0.14321,0.22294,0.28537
7,0.01,0.01,0.36748,0.37062,0.14786,0.24683,0.28598
8,0.01,0.01,0.34630,0.34697,0.18375,0.24337,108.96957
9,0.01,0.01,0.36835,0.36832,0.16309,0.27963,0.25257


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.367916
ABC pre-generator MAE            0.370148
Skip Node weight                 0.157449
ABC GAN MAE                      0.259379
ABC_GAN MAE (skip connection)    0.285676
dtype: float64
-------------------------------------------------------------


In [10]:
# Display Stats Summary Tables 
statsData = []
data = np.array(data)
for i in range(9): 
    data[i] = np.array(data[i])
    statsData.append([gan_median,stats, paramVal[i][0], paramVal[i][1],median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
    #statsData.append([gan_median,stats, paramVal[i][0], paramVal[i][1]])
df = pd.DataFrame(statsData, columns = ['GAN','stats','Variance','Bias','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,GAN,stats,Variance,Bias,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,0.24098,0.37596,1.00,1.00,1.29770,0.27274,0.25438,0.99162
1,0.24098,0.37596,0.10,1.00,0.91058,0.27882,0.25858,0.98961
2,0.24098,0.37596,0.01,1.00,0.84538,0.26266,0.26984,0.98788
3,0.24098,0.37596,1.00,0.10,1.04527,0.29160,0.26009,0.75926
4,0.24098,0.37596,0.10,0.10,0.41087,0.27748,0.26671,0.64002
5,0.24098,0.37596,0.01,0.10,0.38856,0.27486,0.26016,0.57831
6,0.24098,0.37596,1.00,0.01,1.04043,0.27876,0.28270,0.33187
7,0.24098,0.37596,0.10,0.01,0.39034,0.27419,0.28502,0.19690
8,0.24098,0.37596,0.01,0.01,0.37015,0.25938,0.28568,0.15745
